In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%pylab inline
from sklearn.model_selection import train_test_split
#Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

#В файле размещены данные обо всех сессиях всех игроков игры X за последние 90 дней 
#(udid - id игрока, time - дата, count - количество сессий в день, avg_duration - средняя продолжительность сессии 
#в секундах). Постройте на основании этих данных модель прогнозирования ухода игрока. Применяемые методы и 
#инструменты ограничены лишь вашей фантазией и умениями. Главное, что модель в итоге должна уметь прогнозировать 
#для каждого конкретного игрока с какой-либо вероятностью то, что он в ближайшее время покинет проект. 
#Покинувшим проект игроком давайте считать того, кто не заходил в игру уже минимум две недели. 



Populating the interactive namespace from numpy and matplotlib


In [2]:
df = pd.read_csv('TestGameSessions.csv')
df.head()

,udid,time,count,avg_duration
0,104008439,1474588800,2,112
1,104011104,1474588800,2,44
2,104022241,1474588800,2,413
3,104022356,1474588800,2,125
4,104022069,1474592400,5,1561


In [3]:
df['time']= pd.to_datetime(df['time'], unit='s')
df.head()

,udid,time,count,avg_duration
0,104008439,2016-09-23 00:00:00,2,112
1,104011104,2016-09-23 00:00:00,2,44
2,104022241,2016-09-23 00:00:00,2,413
3,104022356,2016-09-23 00:00:00,2,125
4,104022069,2016-09-23 01:00:00,5,1561


In [4]:
df['time'].max()

Timestamp('2016-12-21 23:40:00')

In [5]:
df['churn'] = np.where(df['time'] > '2016-12-08 00:00:00', 1000, 1)
df.head()

,udid,time,count,avg_duration,churn
0,104008439,2016-09-23 00:00:00,2,112,1
1,104011104,2016-09-23 00:00:00,2,44,1
2,104022241,2016-09-23 00:00:00,2,413,1
3,104022356,2016-09-23 00:00:00,2,125,1
4,104022069,2016-09-23 01:00:00,5,1561,1


In [6]:
new_df = df[['udid', 'count','avg_duration', 'churn']].groupby('udid', as_index=False).sum()

In [7]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4267 entries, 0 to 4266
Data columns (total 4 columns):
udid            4267 non-null int64
count           4267 non-null int64
avg_duration    4267 non-null int64
churn           4267 non-null int64
dtypes: int64(4)
memory usage: 166.7 KB


In [8]:
new_df['churn'] = np.where(new_df['churn']>=1000, 0, 1)

In [9]:
new_df.describe()

,udid,count,avg_duration,churn
count,4.267000e+03,4267.000000,4267.000000,4267.000000
mean,1.039534e+08,4.409655,9304.655027,0.825639
std,8.629708e+05,14.182918,20684.575005,0.379464
min,7.438516e+07,1.000000,1.000000,0.000000
25%,1.040112e+08,1.000000,81.000000,1.000000
50%,1.040233e+08,2.000000,685.000000,1.000000
75%,1.040270e+08,4.000000,5242.000000,1.000000
max,1.040312e+08,542.000000,243019.000000,1.000000


In [10]:
train, test = train_test_split(new_df, test_size=0.2)

In [11]:
train.describe()

,udid,count,avg_duration,churn
count,3.413000e+03,3413.000000,3413.000000,3413.000000
mean,1.039418e+08,4.525344,9431.216525,0.823909
std,9.546366e+05,15.414639,21005.534574,0.380954
min,7.438516e+07,1.000000,1.000000,0.000000
25%,1.040114e+08,1.000000,87.000000,1.000000
50%,1.040234e+08,2.000000,720.000000,1.000000
75%,1.040270e+08,4.000000,5240.000000,1.000000
max,1.040312e+08,542.000000,243019.000000,1.000000


In [12]:
test.describe()

,udid,count,avg_duration,churn
count,8.540000e+02,854.000000,854.000000,854.000000
mean,1.039998e+08,3.947307,8798.853630,0.832553
std,2.764985e+05,7.436376,19352.251657,0.373594
min,9.808334e+07,1.000000,1.000000,0.000000
25%,1.040107e+08,1.000000,68.250000,1.000000
50%,1.040232e+08,2.000000,556.000000,1.000000
75%,1.040267e+08,3.750000,5329.750000,1.000000
max,1.040312e+08,96.000000,137493.000000,1.000000


In [13]:
X_train = train.drop(['udid', 'churn'], axis=1) 
Y_train = train['churn']
X_test = test.drop(['udid', 'churn'], axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((3413, 2), (3413,), (854, 2))

In [14]:
# Logistic Regression

logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train)*100, 2)
acc_log

82.86

In [15]:
# Random Forest

random_forest = RandomForestClassifier()
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
acc_random_forest = round(random_forest.score(X_train, Y_train)*100, 2)
acc_random_forest

93.58

In [16]:
# Decision Tree

decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train)*100, 2)
acc_decision_tree

95.31

In [17]:
submission = pd.DataFrame({'udid':test['udid'], 'churn': Y_pred})


In [18]:
submission['churn'].value_counts()

1    728
0    126
Name: churn, dtype: int64

In [19]:
test['churn'].value_counts()

1    711
0    143
Name: churn, dtype: int64